In [0]:
new_york_taxis = spark.read.format("csv")\
                .option("header", "true")\
                .option("inferSchema" , "true")\
                .load("/databricks-datasets/nyctaxi/tripdata/yellow/yellow_tripdata_2019-12.csv.gz")



In [0]:
new_york_taxis.limit(1).display()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
1,2019-12-01T00:26:58.000+0000,2019-12-01T00:41:45.000+0000,1,4.2,1,N,142,116,2,14.5,3.0,0.5,0.0,0.0,0.3,18.3,2.5


Utilize um comando spark para contar quantas linhas tem esse dataframe

In [0]:
import pyspark.sql.functions as F

# df = new_york_taxis.select("VendorID") 

# Não teve diferença no tempo de execução, selecionar 1 ou todas as colunas, para esse caso, não apresentou diferenças.

display(f' O dataframe possui {new_york_taxis.count()} linhas')

' O dataframe possui 6896317 linhas'

Agrupe os dados por número de passageiros e encontre o número de corridas de cada um dos valores.

In [0]:
df = new_york_taxis.select("passenger_count") \
                   .groupBy("passenger_count") \
                   .count() \
                   .withColumnRenamed("count", "total_taxis_trips")

  
display(df)

passenger_count,total_taxis_trips
null,51018
1,4783392
6,147925
3,303007
5,244887
9,16
4,164931
8,18
7,45
2,1075747


DESAFIO: Tente agrupar por dia (1 a 31 de Desembro) e encontre o número de
corridas para cada dia.

In [0]:
df_total_pickup_day = new_york_taxis.select(("tpep_pickup_datetime"), 
                                   F.dayofmonth(("tpep_pickup_datetime")).alias("tpep_pickup_day"))

df_total_pickup_day = df_total_pickup_day.select("tpep_pickup_day") \
                                       .groupBy("tpep_pickup_day") \
                                       .count() \
                                       .withColumnRenamed("count", "total_pickup_per_day")
# display(df_total_pickup_day)                                    

In [0]:
df_total_dropoff_day = new_york_taxis.select(("tpep_dropoff_datetime"), 
                                   F.dayofmonth(("tpep_dropoff_datetime")).alias("tpep_dropoff_day")       
                                       )

df_total_dropoff_day = df_total_dropoff_day.select("tpep_dropoff_day") \
                                       .groupBy("tpep_dropoff_day") \
                                       .count() \
                                       .withColumnRenamed("count", "total_dropoff_per_day")
# df_total_dropoff_day.show()

In [0]:
df_total_trips = df_total_pickup_day.join(df_total_dropoff_day, df_total_pickup_day.tpep_pickup_day == df_total_dropoff_day.tpep_dropoff_day,"inner")

df = df_total_trips.select("tpep_pickup_day", "total_pickup_per_day", "total_dropoff_per_day") \
                                .withColumnRenamed("tpep_pickup_day", "day_of_month") \
                                .withColumn("mean_trips_per_day",(((F.col("total_pickup_per_day") + F.col("total_dropoff_per_day")) / 2))) \
                                .withColumn("Diference_pickup_dropoff", F.col("total_pickup_per_day") - F.col("total_dropoff_per_day"))


display(df)

day_of_month,total_pickup_per_day,total_dropoff_per_day,mean_trips_per_day,Diference_pickup_dropoff
31,169224,169719,169471.5,-495
28,172031,171770,171900.5,261
26,146945,146475,146710.0,470
27,169616,169063,169339.5,553
12,278483,277745,278114.0,738
22,179056,180697,179876.5,-1641
1,182623,182507,182565.0,116
13,278121,277601,277861.0,520
6,274925,274131,274528.0,794
16,236426,236286,236356.0,140
